In [ ]:
!tar -xvf dataset.tar.gz

In [ ]:
%load_ext tensorboard
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import random
import os
import sys
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.applications import ResNet50,ResNet50V2
from tensorflow.keras.optimizers import Adam




input_shape=(128,128,3)
batch_size = 20
epochs = 20



def build_model():
    model = Sequential([
        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', 
                  input_shape=input_shape),
        BatchNormalization(),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, kernel_size=(3, 3), activation='relu'),
        BatchNormalization(),        
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, kernel_size=(3, 3), activation='relu'),
        BatchNormalization(),    
        MaxPooling2D(pool_size=(2, 2)),   
        
        Flatten(),
        
        Dense(1024, activation='relu'),
        
        Dense(512, activation='relu'),
        
        Dense(1, activation='sigmoid')
        
    ])
    opt = Adam(lr=1e-6)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model


def training():
    # define model
    model = build_model()
    # create data generator
    train_datagen = ImageDataGenerator(
        rotation_range=15,
        rescale=1.0/255.,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1)

    validation_datagen = ImageDataGenerator( rescale = 1.0/255.)
    test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

    # prepare iterators
    train_generator = train_datagen.flow_from_directory('output/train/',
                                                 class_mode='binary',
                                                 batch_size=batch_size, 
                                                 target_size=input_shape[:2])
    test_generator = test_datagen.flow_from_directory('output/test/',
                                               class_mode='binary', 
                                               batch_size=batch_size, 
                                               target_size=input_shape[:2])
    validation_generator = validation_datagen.flow_from_directory('output/val/',
                                                        class_mode='binary',
                                                        batch_size=batch_size,
                                                        target_size=input_shape[:2])
    steps_per_epoch = train_generator.n // batch_size
    validation_steps = validation_generator.n // batch_size
    earlystop = EarlyStopping(monitor='val_accuracy', 
                              min_delta=0.05, 
                              patience=10, 
                              verbose=1, 
                              mode='auto')
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                                patience=10,
                                                verbose=1,
                                                factor=0.5,
                                                min_lr=0.00001,
                                                mode='auto'
                                               )
    checkpoint_filepath = 'weights/'
    model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    callbacks_list = [earlystop, learning_rate_reduction,model_checkpoint_callback]
    # fit model
    history = model.fit(train_generator, 
                    steps_per_epoch=steps_per_epoch,
                    validation_data=validation_generator, 
                    validation_steps=validation_steps, 
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks_list
                   )
    # evaluate model
    _, acc = model.evaluate(test_generator, steps=len(test_generator), verbose=1)
    print('> %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)

def summarize_diagnostics(history):
	
    acc      = history.history['accuracy']
    val_acc  = history.history['val_accuracy']
    loss     = history.history['loss']
    val_loss = history.history['val_loss']

    epochs   = range(1,len(acc)+1,1)
    plt.subplot(211)
    plt.title('Classification Accuracy')
    plt.plot ( epochs,     acc, 'r--', label='Training acc'  )
    plt.plot ( epochs, val_acc,  'b', label='Validation acc')
    plt.title ('Training and validation accuracy')
    plt.ylabel('acc')
    plt.xlabel('epochs')

    plt.legend()
    plt.figure()
    
    plt.subplot(212)
    plt.title('Cross Entropy Loss')
    plt.plot ( epochs,     loss, 'r--' )
    plt.plot ( epochs, val_loss ,  'b' )
    plt.title ('Training and validation loss'   )
    plt.ylabel('loss')
    plt.xlabel('epochs')

    plt.legend()
    plt.figure()
    plt.show()
if __name__=='__main__':
    training()
    

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 128, 128, 64)      1792      
_________________________________________________________________
batch_normalization_30 (Batc (None, 128, 128, 64)      256       
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 126, 126, 64)      36928     
_________________________________________________________________
batch_normalization_31 (Batc (None, 126, 126, 64)      256       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 63, 63, 128)       73856     
______________________________